In [2]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("","","")

Found the config file in: /config.json


In [3]:
dataset_parent_dir = "./data"
dataset_name = "lemon_Dataset"
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

In [4]:


training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")



In [5]:


my_training_data_input = Input(type=AssetTypes.MLTABLE,path=training_mltable_path)
my_validation_data_input = Input(type=AssetTypes.MLTABLE,path=validation_mltable_path)



**
Sent to Training**


In [6]:


compute_name = "gpl-cluster-nc6sv3"
def create_cluster_computer(compute_name):
    try:
        _ = ml_client.compute.get(compute_name)
        print("Found Existing compute target")
    except:
        print("Creating a new Computer target")
        conpute_config = AmlCompute(
            name=compute_name,
            type="amlcompute",
            size="Standard_NC6s_v3",
            idle_time_before_scale_down=120,
            min_instances=0,
            max_instances=4
        )
        ml_client.begin_create_or_update(conpute_config).result()
        print("Done")

create_cluster_computer(compute_name)



Found Existing compute target




**Create image classification job**


In [7]:


exp_name = "dl-lemon-objects"

image_classfication_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name='label',
    primary_metric="accuracy",
    tags={"exp-name":"Job for image classification with lemon objetcs"}
)

image_classfication_job.set_limits(max_trials=4, max_concurrent_trials=2)

#returned_job = ml_client.jobs.create_or_update(image_classfication_job)




tuning de hiperparámetros


In [8]:


from azure.ai.ml.automl import SearchSpace
from azure.ai.ml.sweep import (Choice,Uniform,BanditPolicy,)



In [9]:
# eleccion modelos a revisar
tun_1 = SearchSpace( model_name=Choice(["vitb16r224", "yolov5"]),learning_rate=Uniform(0.001, 0.01),number_of_epochs=Choice([15, 30]), )
tun_2 = SearchSpace( model_name=Choice(["seresnext", "resnet50"]),learning_rate=Uniform(0.001, 0.01),layers_to_freeze=Choice([0, 2]),)
tunning = [tun_1, tun_2]

image_classfication_job.extend_search_space(tunning)

In [10]:


# toma de muestras random
# politicas de terminacion
limit_job = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6)
image_classfication_job.set_sweep(sampling_algorithm="Random", early_termination=limit_job,)



In [11]:
returned_job = ml_client.jobs.create_or_update(image_classfication_job)

In [12]:
returned_job.name

'sweet_bell_qc73gvv9m6'

In [9]:
import mlflow

MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/ws_ml_rcf


'wheat_ship_yr2b9njcm6'